## Necessary components

### Import libraries

In [ ]:
from trident_token_library_wrapper import PyTridentTokenLibrary
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests 
import json

### Define variables

In [ ]:
# Name of the Key Vault
nameOfKeyVault = 'xxxx' # Name of the Key Vault
# Names of the secrets saved in Key Vault
tenantId_SecretName = 'xxxx' # Name for Tenant ID
clientId_SecretName = 'xxxx'   # Name for Client ID of Service Principal
clientSecret_SecretName = 'xxxx' # Name for Client Secret of Service Principal

# Base URLS for Power BI, Fabric and Key Vault
pbi_Uri = 'https://api.powerbi.com/v1.0/myorg/'
fabric_Uri = 'https://api.fabric.microsoft.com/v1/'
keyvault = f'https://{nameOfKeyVault}.vault.azure.net/'

### Define functions to get key vault secrets, API bearer token and JSON responses

In [ ]:
def get_secret_from_keyvault(secret_name):
    access_token=PyTridentTokenLibrary.get_access_token("keyvault")
    return PyTridentTokenLibrary.get_secret_with_token(keyvault,secret_name,access_token)

def get_bearer_token():
    tenant_id = get_secret_from_keyvault(tenantId_SecretName)
    client_id = get_secret_from_keyvault(clientId_SecretName)
    client_secret = get_secret_from_keyvault(clientSecret_SecretName)
    url = "https://login.microsoftonline.com/" + tenant_id + "/oauth2/token"
    data = "grant_type=client_credentials&client_id=" + client_id + "&client_secret=" + client_secret + "&resource=https://analysis.windows.net/powerbi/api"  
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.post(url, headers=headers, data=data)
    return response.json()["access_token"]

def get_response_json(fullurl, method, data , payload_object):
    bearer_token = get_bearer_token()
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {bearer_token}'}
    response = requests.request(method, fullurl, headers=headers, data=data)
    if payload_object == "":
        return [response.text]
    else:
        return response.json()[payload_object]

## Functions

### Admin

#### Get all capacities in tenant

In [41]:
source = get_response_json(pbi_Uri+'admin/capacities',"GET","","value")
df = spark.read.option("multiline","true").json(sc.parallelize(source))
dfDrop = df.drop('admins')

writeToLake = dfDrop.write.mode("overwrite").format("delta").save("Tables/Capacities")

#### Get all capacity admins 

In [ ]:
source = get_response_json(pbi_Uri+'admin/capacities',"GET", "", "value")
df = spark.read.option("multiline","true").json(sc.parallelize(source))
select = df.select(df.id, explode("admins").alias("admins"))

writeToLake = select.write.mode("overwrite").format("delta").save("Tables/CapacityAdmins")

#### Get all Domains

In [ ]:
source = get_response_json(pbi_Uri+'admin/domains',"GET", "", "domains")
df = spark.read.json(sc.parallelize(source))

writeToLake = df.write.mode("overwrite").format("delta").save("Tables/Domains")

#### Get Tenant Settings

In [ ]:
data = get_response_json(fabric_Uri+'admin/tenantsettings',"GET", "", "tenantSettings")

schema = StructType([
    StructField("enabled",BooleanType(),True),
    StructField("title",StringType(),True),
    StructField('canSpecifySecurityGroups',BooleanType(),True),
    StructField('settingName',StringType(),True),
    StructField('tenantSettingGroup',StringType(),True),
    StructField('enabledSecurityGroups',ArrayType(StructType([StructField('graphId', StringType(),True),StructField('name', StringType(),True)])),True)
  ])

df = spark.createDataFrame(spark.sparkContext.parallelize(data),schema)

writeToLake = df.write.mode("overwrite").format("delta").save("Tables/TenantSettings")


#### Get Imports

In [ ]:
source = get_response_json(pbi_Uri+'admin/imports',"GET", "", "value")
df = spark.read.json(sc.parallelize(source)).select(["id","importState","createdDateTime","updatedDateTime","name","connectionType","source","datasets","reports","dataflows"])

writetolake = df.write.mode("overwrite").format("delta").save("Tables/Imports")

#### Get Refreshables

In [ ]:
source = get_response_json(pbi_Uri+'admin/capacities/refreshables',"GET", "", "")
refreshablesDf = spark.read.option("inferSchema", "true").option("multiLine", "true").json(sc.parallelize(source))
parseRefreshablesDf = refreshablesDf.withColumn("refreshables", explode("value")).select("refreshables")

parser = parseRefreshablesDf.select(expr("refreshables.name"),expr("refreshables.id"),expr("refreshables.kind"),expr("refreshables.refreshSchedule.enabled"),expr("refreshables.refreshSchedule.localTimeZoneId"),expr("refreshables.refreshSchedule.times"),expr("refreshables.refreshSchedule.notifyOption"),expr("refreshables.refreshSchedule.days"),expr("refreshables.lastRefresh.startTime"),expr("refreshables.lastRefresh.endTime"),expr("refreshables.lastRefresh.status"),expr("refreshables.lastRefresh.requestId"),expr("refreshables.lastRefresh.refreshType"),expr("refreshables.configuredBy"))
writetolake = parser.write.mode("overwrite").format("delta").save("Tables/Refreshables")

### Execute Query

#### query a dataset

In [ ]:
datasetId = 'xxxxxxxxxxxx'
DAXQuery = 'EVALUATE Date'
body = '{"queries":[{"query":"' + DAXQuery + '"}],"serializerSettings":{"incudeNulls":true}}'
source = get_response_json('https://api.powerbi.com/v1.0/myorg/datasets/' + datasetId + '/executeQueries',"POST", body, "results")
print(source)